In [3]:
import os
import random

import cv2
from ultralytics import YOLO

from deepsorttr import Tracker


video_path = os.path.join('videos', '1.mp4')
video_out_path = os.path.join('videos', '1_out_3.mp4')

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
print(frame.shape)
cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

model = YOLO("best.pt")
model.fuse()

tracker = Tracker()

colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]

detection_threshold = 0.3
while ret:

    results = model(frame, verbose=False)

    for result in results:
        detections = []
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)
            class_id = int(class_id)
            if score > detection_threshold:
                detections.append([x1, y1, x2, y2, score])

        tracker.update(frame, detections)

        for track in tracker.tracks:
            bbox = track.bbox
            x1, y1, x2, y2 = bbox
            track_id = track.track_id

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 3)
            # cv2.putText(frame, "ID: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    cap_out.write(frame)
    ret, frame = cap.read()
    
    # cv2.imshow("Tracking", frame)

    # # Press 'q' to exit the loop
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap.release()
cap_out.release()
cv2.destroyAllWindows()

(1440, 2160, 3)


Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
